<img src="https://avatars.githubusercontent.com/u/74911464?s=200&v=4"
     alt="OpenEO Platform logo"
     style="float: left; margin-right: 10px;" />
# Forest canopy cover mapping using openEO Platform
## Demo at the ESA Living planet symposium
### 27.05.2022


The approach subsequently presented has the objective to derive the fractional canopy cover (FCC) in forest based on Planetscope very-high resolution data as well as Sentinel-2 data. It includes several processing steps involving the calculation and transformation of datacubes, as well as training and prediction of FCC with a random forest regression.

This Python notebook utilizes the [OpenEO Python client](https://github.com/Open-EO/openeo-python-client) to construct openEO jobs in form of JSON files sent as http requests to the back-ends. This will be accomplished in three main steps:
- Computation of the target variable (FCC) from VHR data cubes
- Training of the model
- Prediction of the model

## Requirements

First the necessary python modules must be loaded

In [56]:
import openeo
from openeo.processes import median, sd
import numpy as np
import geopandas as gpd
import xarray as xr
import matplotlib.pyplot as plt
from shapely.geometry.polygon import Polygon
import json
from openeo.rest.datacube import PGNode, THIS

## Connection

Connect to the OpenEO back-end using the OpenEO client

In [7]:
connection = openeo.connect("openeo.cloud")

Authenticate via EGI Check-in (OpenID Connect)

In [8]:
connection.authenticate_oidc("egi")
connection

Authenticated using refresh token.


<Connection to 'https://openeocloud.vito.be/openeo/1.0.0/' with OidcBearerAuth>

## 1. Target Variable for regression

The VHR data from `PLANETSCOPE` - the base for the target variable of our regression - is stored in a dedicated **OpenEO collection**. 
The `PLANETSCOPE`collection is commercial data that has been acquired specifically for this project. It is not openly accessible to all users.

In [9]:
connection.describe_collection("PLANETSCOPE")

{'cube:dimensions': {'bands': {'type': 'bands',
   'values': ['B1',
    'B2',
    'B3',
    'B4',
    'UDM',
    'UDM2_Clear',
    'UDM2_Snow',
    'UDM2_Shadow',
    'UDM2_LightHaze',
    'UDM2_HeavyHaze',
    'UDM2_Cloud',
    'UDM2_Confidence',
    'dataMask']},
  't': {'extent': ['2016-01-01T00:00:00Z', None], 'type': 'temporal'},
  'x': {'axis': 'x',
   'extent': [-180, 180],
   'reference_system': {'$schema': 'https://proj.org/schemas/v0.2/projjson.schema.json',
    'area': 'World',
    'base_crs': {'coordinate_system': {'axis': [{'abbreviation': 'Lat',
        'direction': 'north',
        'name': 'Geodetic latitude',
        'unit': 'degree'},
       {'abbreviation': 'Lon',
        'direction': 'east',
        'name': 'Geodetic longitude',
        'unit': 'degree'}],
      'subtype': 'ellipsoidal'},
     'datum': {'ellipsoid': {'inverse_flattening': 298.257223563,
       'name': 'WGS 84',
       'semi_major_axis': 6378137},
      'name': 'World Geodetic System 1984',
      'type': 'GeodeticReferenceFrame'},
     'name': 'WGS 84'},
    'bbox': {'east_longitude': 180,
     'north_latitude': 90,
     'south_latitude': -90,
     'west_longitude': -180},
    'coordinate_system': {'axis': [{'abbreviation': 'E',
       'direction': 'east',
       'name': 'Easting',
       'unit': 'metre'},
      {'abbreviation': 'N',
       'direction': 'north',
       'name': 'Northing',
       'unit': 'metre'}],
     'subtype': 'Cartesian'},
    'id': {'authority': 'OGC', 'code': 'Auto42001', 'version': '1.3'},
    'name': 'AUTO 42001 (Universal Transverse Mercator)',
    'type': 'ProjectedCRS'},
   'type': 'spatial'},
  'y': {'axis': 'y',
   'extent': [-90, 90],
   'reference_system': {'$schema': 'https://proj.org/schemas/v0.2/projjson.schema.json',
    'area': 'World',
    'base_crs': {'coordinate_system': {'axis': [{'abbreviation': 'Lat',
        'direction': 'north',
        'name': 'Geodetic latitude',
        'unit': 'degree'},
       {'abbreviation': 'Lon',
        'direction': 'east',
        'name': 'Geodetic longitude',
        'unit': 'degree'}],
      'subtype': 'ellipsoidal'},
     'datum': {'ellipsoid': {'inverse_flattening': 298.257223563,
       'name': 'WGS 84',
       'semi_major_axis': 6378137},
      'name': 'World Geodetic System 1984',
      'type': 'GeodeticReferenceFrame'},
     'name': 'WGS 84'},
    'bbox': {'east_longitude': 180,
     'north_latitude': 90,
     'south_latitude': -90,
     'west_longitude': -180},
    'coordinate_system': {'axis': [{'abbreviation': 'E',
       'direction': 'east',
       'name': 'Easting',
       'unit': 'metre'},
      {'abbreviation': 'N',
       'direction': 'north',
       'name': 'Northing',
       'unit': 'metre'}],
     'subtype': 'Cartesian'},
    'id': {'authority': 'OGC', 'code': 'Auto42001', 'version': '1.3'},
    'name': 'AUTO 42001 (Universal Transverse Mercator)',
    'type': 'ProjectedCRS'},
   'type': 'spatial'}},
 'datasource_type': 'byoc-ID',
 'description': "PlanetScope is one of the satellite constellation operated by Planet. PlanetScope satellite constellation consists of more than 130 small satellites called Doves. Each Dove satellite is a CubeSat made of three cubic units and thus measures only 10 cm x 10 cm x 30 cm. The satellites are launched in groups, which constantly improves mission's characteristics such as revisit times, spatial and spectral resolutions. The constellation is constantly on and does not require an acquisition planning. PlanetScope is commercial data and has to be ordered by the user",
 'extent': {'spatial': {'bbox': [[-180, -90, 180, 90]]},
  'temporal': {'interval': [['2016-01-01T00:00:00Z', None]]}},
 'id': 'PLANETSCOPE',
 'keywords': ['sentinel hub', 'PlanetScope', 'vhr', 'commercial data'],
 'license': 'various',
 'links': [],
 'providers': [{'description': '',
   'name': 'Sentinel Hub',
   'roles': ['processor'],
   'url': 'https://docs.sentinel-hub.com/api/latest/data/planet-scope/'},
  {'description': '',
   'name': 'Planet',
   

It is required to use a specific FeatureFlag denominated **BYOC** (Bring your own collection). It is read from a textfile that cannot be shared publicly.  This means that this notebook is not reproducible for all users without the BYOC code for this data cube. 

In [10]:
byoc_id_file = "extdata/byoc.txt"
byoc_id  = open(byoc_id_file,"r").read().splitlines()[0]

### External Data

In order to process all the test sites a shapefile is read containing the boundaries of the smaller test areas. We applied a 0.0001° buffer in order to obtain all Pixels in the test sites

In [11]:
shp_path = "./resources/UC8/vector_data/SuitableSitesVHR_selected_country.shp"
aoi_geometries = gpd.read_file(shp_path)
aoi_buffered_geometries = aoi_geometries.buffer(0.0001)
aoi_buffered_geometries

/tmp/ipykernel_7591/505307699.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  aoi_buffered_geometries = aoi_geometries.buffer(0.0001)


0      POLYGON ((7.79031 50.30665, 7.79030 50.30665, ...
1      POLYGON ((11.89106 43.92545, 11.89105 43.92545...
2      POLYGON ((7.23700 49.18974, 7.23699 49.18974, ...
3      POLYGON ((7.51359 49.47334, 7.51358 49.47334, ...
4      POLYGON ((16.97810 46.88940, 16.97809 46.88940...
                             ...                        
145    POLYGON ((9.95005 47.03182, 9.95004 47.03182, ...
146    POLYGON ((12.25466 44.82748, 12.25465 44.82748...
147    POLYGON ((6.84307 43.52251, 6.84306 43.52251, ...
148    POLYGON ((12.08926 44.18149, 12.08925 44.18149...
149    POLYGON ((4.69950 43.55341, 4.69949 43.55341, ...
Length: 150, dtype: geometry

In [12]:
aois_bounds = aoi_buffered_geometries.bounds.to_numpy()
print("Bounds of the first considered AOI in lat/lon (WGS84): ",aois_bounds[0])

Bounds of the first considered AOI in lat/lon (WGS84):  [ 7.79003931 50.30665141  7.79601901 50.310554  ]


### Connect to VHR data

In this exemplary Notebook the target variable is calculated for the first test site in Germany for the year 2018

In [13]:
aoi0_bounds = aois_bounds[0]
year = 2018

It might be difficult to use  to properly distinguish between forests and other land cover classes, especially those that also represent vegetation such as crops or grasslands. *Yang et al. (2019)* showed a nice representation of standardized forest NDVI signatures that nicely show the seasonal trend within a year based on different forest types. There are several time intervals of interest when generating a forest mask. Due to the high frequency of revisits by the `PLANETSCOPE` satellites each time interval is represented multiple times with several images. The time intervals of interest are:
- **summer_time**: Time of the maximum productivity of vegetation from mid April to mid September
- **winter_time**: Time of the minimum productivity of vegetation from mid November to mid February
- **annual_time**: The whole year range
- **total_time**: Takes into account the **annual_year** as well as some shoulder values in Winter (until mid-february the following year)

In [14]:
summer_time =  [np.datetime64(str(year)+"-04-15"),np.datetime64(str(year)+"-09-15")]
winter_time =  [np.datetime64(str(year)+"-11-15"),np.datetime64(str(year+1)+"-02-15")]
annual_time =  [np.datetime64(str(year)+"-01-01"),np.datetime64(str(year)+"-12-31")]
total_time  =  [np.datetime64(str(year)+"-01-01"),np.datetime64(str(year+1)+"-02-15")]

The `PLANETSCOPE` data is loaded primarily for **total_time** before the NDVI is calculated. Afterwards a clear-sky mask is applied to exclude all Pixel that are altered by clouds, haze and other atmospheric effects.

In [15]:
plnt = connection.load_collection(
    collection_id  = "PLANETSCOPE",
    spatial_extent = {"west": aoi0_bounds[0], 
                      "south": aoi0_bounds[1], 
                      "east": aoi0_bounds[2], 
                      "north": aoi0_bounds[3]},
    temporal_extent= [str(total_time[0]), str(total_time[1])]
    )
plnt._pg.arguments['featureflags'] = {'byoc_collection_id': byoc_id}

plnt_ndvi = plnt.ndvi(nir="B4",red="B3")
mask      = plnt.band("UDM2_Clear").apply(lambda x: x.neq(1))
plnt_ndvi_msk = plnt_ndvi.mask(mask=mask)

### Create Seasonal masks

In this step the **summer_time** is applied to the data using the `filter_temporal` process. For analysing the whole summer period based on `median` and `sd` metrics, the `reduce_dimension` process is used on the *t* dimension.

We calculate two indicators important for the mask generation:
- **s_msk_med_hig**: Median summer NDVI above 0.6
- **s_msk_sd_low**: Summer NDVI standard deviation below 0.1

In [16]:
summer_time =  [np.datetime64(str(year)+"-04-15"),np.datetime64(str(year)+"-09-15")]
plnt_summer =  plnt_ndvi_msk.filter_temporal(extent= [str(summer_time[0]), str(summer_time[1])])

s_msk_med     = plnt_summer.reduce_dimension(dimension='t',reducer=median)
s_msk_med_hig = s_msk_med > 0.6

s_msk_sd      = plnt_summer.reduce_dimension(dimension='t',reducer=sd)
s_msk_sd_low  = s_msk_sd < 0.1

In this step the **winter_time** is applied to the data using the `filter_temporal` process. For analysing the whole summer period based on `median` metric, the `reduce_dimension` process is used on the *t* dimension.

Here we calculate two indicators important for the mask generation:
- **w_msk_med_hig**: Median summer NDVI above 0.6
- **w_msk_med_low**: Median summer NDVI between 0 and 0.4

In [17]:
winter_time =  [np.datetime64(str(year)+"-11-15"),np.datetime64(str(year+1)+"-02-15")]
plnt_winter =  plnt_ndvi_msk.filter_temporal(extent= [str(winter_time[0]), str(winter_time[1])])

w_msk_med     = plnt_winter.reduce_dimension(dimension='t',reducer=median)
w_msk_med_hig = w_msk_med > 0.6

w_low_upper   = w_msk_med < 0.4
w_low_lower   = w_msk_med > 0
w_msk_med_low = w_low_upper * w_low_lower

In this step the **year_time** is applied to the data using the `filter_temporal` process. For analysing the whole summer period based on `median` metric, the `reduce_dimension` process is used on the *t* dimension.

Here we calculate two indicators important for the mask generation:
- **y_msk_med_hig**: Median yearly NDVI above 0.6

In [18]:
annual_time =  [np.datetime64(str(year)+"-01-01"),np.datetime64(str(year)+"-12-31")]
plnt_year   =  plnt_ndvi_msk.filter_temporal(extent= [str(annual_time[0]), str(annual_time[1])])

y_msk_med     = plnt_year.reduce_dimension(dimension='t',reducer=median)
y_msk_med_hig = y_msk_med > 0.6

### Create thematic masks

Based on the **seasonal_masks** also thematic ones are created:

- **f_evergreen_mask**: A mask with high summer and high winter values typically for evergreen (conifer) forests
- **f_deciduous_mask**: A mask with high summer and very low winter values typically hinting at deciduous forests
- **f_mixed_mask**: Mixed forests typically have a high yearly and high summer value but are the most difficult to spot. This mask sustains the other to exlude other vegetation types

In [19]:
f_evergreen_mask  = s_msk_med_hig * w_msk_med_hig
f_deciduous_mask  = s_msk_med_hig * w_msk_med_low
f_mixed_mask      = y_msk_med_hig * s_msk_med_hig

Finally the masks are being combined to a final forest canopy cover mask with th following criteria:
- **cmask_forest**: Estimates whether it is either an evergreen, deciduous forest or mixed (or multiple of those)
- **cmask_forest_error**: Excludes Pixel with a high standard deviation in summer. These are often hinting at management processes typical for crops and grassland vegetation

In [20]:
cmask_forest = f_evergreen_mask + f_deciduous_mask + f_mixed_mask
cmask_forest = cmask_forest > 0
cmask_forest_sd = cmask_forest * s_msk_sd_low

### Resample

The target variable will be resampled to 60m. The output pixels will be the result of the average over 20 x 20 (400) Planet pixels, since the Planet pixel size is 3 meters.

The 60m x 60m unit area corresponds to 6x6 (36) Sentinel-2 pixels with 10 meters resolution and 3x3 (9) Sentinel-1 pixels with 20m resolution.

This allows for a more stable estimation of the target variable during the random Forest regression

In [21]:
test_res = cmask_forest_sd.resample_spatial(resolution=60,method="average")
res_save = test_res.save_result(format="NetCDF")

In [22]:
job = res_save.create_job(title = "VH0_result_Resample60_average_lps")
job.start_job()

In [24]:
# job = "vito-17416247-bedb-44d8-8a95-99d633ad0470"
job

<RESTJob job_id='vito-17416247-bedb-44d8-8a95-99d633ad0470'>

In [ ]:
results = job.get_results()
results.download_files("./resources/UC8/canopy_cover_masks/AOI0/")

### Conversion to Vector

After this last step the NetCDF containing the resampled information about the fractional canopy cover is converted to a vector-based GeoJSON file where each pixel is represented by a different Polygon. This allows to use it as Input file for the extracting the training samples needed for the random Forest regression

## 2. Training of the model

In a first step we have to connect to a different OpenEO environment. As some functionalities are currently under heavy development they are not yet deployed on the live back-end. This will occur soon after the Living planet symposium when all the remaining uncertainties were tackled

In [25]:
conn = openeo.connect("https://openeo-dev.eodc.eu/v1.0")
#conn = openeo.connect("https://openeo.eodc.eu/v1.0")
conn.authenticate_oidc("egi")
#conn = connection
conn

Authenticated using refresh token.


<Connection to 'https://openeo-dev.eodc.eu/v1.0' with OidcBearerAuth>

### Target and Predictors

The total area covers around 150 AOIs. As the aggregate spatial process is computationally intensive, we distributed the 150 AOIs into 5 jobs in order to create 5 vector cubes using the aggregate spatial process. Afterwards we can merge the 5 vector cubes together. s
At the moment this process is still in development to allow a user to bypass the splitting of the data. Currently we are working on an efficient way to tile the vectors similarly to the data cubes in order to reduce the computation for single jobs. In future there is no need to create multiple jobs for the aggregate spatial operation

In [26]:
import numpy as np

job_ids = []
index = [list(np.arange(0,30)), list(np.arange(30,60)), list(np.arange(60,90)), list(np.arange(90,120)), list(np.arange(120,150))]
no_index = [37, 78, 82]

for l in index:
    merged_data = None
    for i in l:
        aoi_URL = f"https://raw.githubusercontent.com/openEOPlatform/UC8_auxdata/master/vector_data/target_canopy_cover_60m_equi7/target_canopy_cover_equi7_60m_AOI{i}.geojson"
        if i not in no_index:
            aoi_gdf = gpd.read_file(aoi_URL)
            aoi_gdf = aoi_gdf.to_crs('EPSG:4326')
            aoi_bounds = aoi_gdf.total_bounds

            collection      = "boa_sentinel_2"
            spatial_extent  = {"east": aoi_bounds[2],
                               "north": aoi_bounds[3],
                               "south": aoi_bounds[1],
                               "west": aoi_bounds[0]}
            temporal_extent = ["2018-05-01", "2018-09-01"]
            # bands           = ["B02","B03","B04","B05","B06","B07","B08","B11","B12"]
            bands           = ["band_1","band_2","band_3","band_4","band_7","band_9","band_10"]


            boa_sentinel_2_cube = conn.load_collection(
                collection_id   = collection,
                spatial_extent  = spatial_extent,
                temporal_extent = temporal_extent,
                bands = bands
                )

            boa_sentinel_2_cube_reduced = boa_sentinel_2_cube.reduce_dimension(dimension="t",reducer=median)

            aoi_vector_cube = PGNode("load_vector_cube",{"URL":aoi_URL})
            sentinel_2_predictors = boa_sentinel_2_cube_reduced.process("aggregate_spatial",{"data":THIS,"target_dimension":"result","geometries":aoi_vector_cube,"reducer":"mean"})
            full_training_data_cube = sentinel_2_predictors.process("merge_cubes",{"cube1":THIS,"cube2":aoi_vector_cube})
            if merged_data is None:
                merged_data = full_training_data_cube
            else:
                merged_data = merged_data.process("merge_cubes",{"cube1":THIS,"cube2":full_training_data_cube})

    save_vector_cube = merged_data.process("save_vector_cube",{"data":THIS})
        
    job = save_vector_cube.create_job(title=f"UC8_AOI_{l}_vector_cube")
    job_id = job.job_id
    job_ids.append(job_id)

Afterwards we can start the Jobs iteratively

In [27]:
from time import sleep
for jb in job_ids:
    job = conn.job(jb)
    job.start_job()
    print('Job ', jb, ' has been started. ')
    sleep(120)

Job  jb-a55da16f-dc17-4e02-b54b-f902df3a3c40  has been started. 
Job  jb-34adbfac-d973-4c41-8278-b5306a881642  has been started. 
Job  jb-de4b811b-e8f9-4492-bc7f-5e5bdc4757b0  has been started. 
Job  jb-71c38396-014f-4ffc-b40c-6e6bbbd68e2f  has been started. 
Job  jb-399de5cd-3ecc-43ba-9978-612295c16e8f  has been started. 


And check the status

In [28]:
for jb in job_ids:
    job = conn.job(jb)
    job_description = job.describe_job()
    print("Batch job with id: ",jb, ' is ',job_description['status'])

Batch job with id:  jb-a55da16f-dc17-4e02-b54b-f902df3a3c40  is  finished
Batch job with id:  jb-34adbfac-d973-4c41-8278-b5306a881642  is  finished
Batch job with id:  jb-de4b811b-e8f9-4492-bc7f-5e5bdc4757b0  is  finished
Batch job with id:  jb-71c38396-014f-4ffc-b40c-6e6bbbd68e2f  is  finished
Batch job with id:  jb-399de5cd-3ecc-43ba-9978-612295c16e8f  is  finished


Each of the jobs include a wide range of different openEO processes that are being called sequentially. At the moment the spatial aggregation of one AOI is done in less than a minute. In future - when all different Jobs are unified in a single one and a tiling will be applied - the aggregation of raster information in vectors is going to be much faster.
Now we can take a glimpse at one Job

In [29]:
# job= "jb-399de5cd-3ecc-43ba-9978-612295c16e8f"
job

<RESTJob job_id='jb-399de5cd-3ecc-43ba-9978-612295c16e8f'>

Once the Jobs are finished we can use the *merge_cubes* process to bring all the aggregated vector files together in one file as base for the training of the machine learning model

In [31]:
merge = None
for jb in job_ids:
    if merge is None:
        load_vector_cube = full_training_data_cube.process("load_vector_cube",{"job_id":jb})
        merge = load_vector_cube
    else:
        load_vector_cube = full_training_data_cube.process("load_vector_cube",{"job_id":jb})
        merge = merge.process("merge_cubes",{"cube1":THIS,"cube2":load_vector_cube})


### Training

The random forest model is trained with some basic arguments that allow to modify some of the aspects in the training. In our model we use all the bands of Sentinel-2 to train 100 trees. The process *fit_regr_random_forest* is used for training

In [32]:
rf_arguments = {
    "data": merge,
    "max_variables" : None, # Variable used at a node
    "num_trees": 100, # Number of trees in a rf model
    "seed": 0,        # Random Number seed for reproducibility
    # "predictors_vars": ["B02","B03","B04","B05","B06","B07","B08","B11","B12"], # Name of predictors
    "predictors_vars":["band_1","band_2","band_3","band_4","band_7","band_9","band_10"],
    "target_var": "target_canopy_cover_x" # Name of target variable
}

rf_model = merge.process("fit_regr_random_forest", rf_arguments) # fit the model

Afterwards the model can be saved...

In [33]:
rf_model_saved = rf_model.process("save_ml_model",{"model":THIS})

...and the Job can be started

In [34]:
job_train = rf_model_saved.create_job(title="UC8_all_fit_rf_regr_lps")
job_train.start_job()

In [36]:
# job_train = "jb-bd7703ac-9309-4665-9b10-600dceae44d7"
job_train

<RESTJob job_id='jb-bd7703ac-9309-4665-9b10-600dceae44d7'>

## 3. Prediction

For demonstration purposes we are going to predict an area around the Adamello natural park based on the predictors for 2018 

In [37]:
#spatial_extent  =  {"west":10.454955,
#                    "east":10.737297,
#                    "south":46.102185,
#                    "north":46.133657}
spatial_extent  =  {"west":10.45,
                    "east":10.74,
                    "south":46.10,
                    "north":46.14}

The model will be applied to this area based on the same collections and bands we have used for the training of the model. As the predictors are universal for 2018 we base the prediction on the summer period using the *reduce_dimension* process. Being in an early stage of development, future developments will include diverse predictors based on time and therefore also the prediction will change allowing more features

In [38]:
collection      = "boa_sentinel_2"
temporal_extent = ["2018-05-01", "2018-09-01"]
# bands           = ["B02","B03","B04","B08"]
bands           = ["band_1","band_2","band_3","band_4","band_7","band_9","band_10"]

boa_sentinel_2_cube = conn.load_collection(
    collection_id   = collection,
    spatial_extent  = spatial_extent,
    temporal_extent = temporal_extent,
    bands = bands
    )
boa_sentinel_2_cube_reduced = boa_sentinel_2_cube.reduce_dimension(dimension="t",reducer=median)

Predict the model based on a saved model created in the step beforehand

In [39]:
model_jb = "jb-561da48a-359e-43c9-b45e-7d7e8524a1fc"
fractional_canopy_cover = boa_sentinel_2_cube_reduced.predict_random_forest(dimension="bands", model=model_jb)

And now the result can be saved...

In [40]:
fractional_canopy_cover_netcdf = fractional_canopy_cover.save_result(format="netCDF")

...before the job is executed

In [41]:
job_pred = fractional_canopy_cover_netcdf.create_job(title="UC8_predict_rf_adamello_lps")
job_pred.start_job()

The job can be visualized as follows:

In [43]:
# job_pred = "jb-7df5b3c3-c0fe-492c-b161-5a7f824c182a"
job_pred

<RESTJob job_id='jb-7df5b3c3-c0fe-492c-b161-5a7f824c182a'>

### Plotting

Finally we can download of the prediction in Netcdf format to our hard disks. The resulting structure can be loaded in an xarray data cube an finally plotted

In [44]:
results = job_pred.get_results()
results.download_files("./resources/UC8/canopy_cover_prediction/")

[PosixPath('resources/UC8/canopy_cover_prediction/out_2022_05_18T11_48_05_EU010M_E047N014T1'),
 PosixPath('resources/UC8/canopy_cover_prediction/out_2022_05_18T11_48_05_EU010M_E048N014T1'),
 PosixPath('resources/UC8/canopy_cover_prediction/job-results.json')]

In [ ]:
![]() # Bild